In [1]:
pip install nltk

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyttsx3

In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


training = pd.read_csv('Training.csv')
testing= pd.read_csv('Testing.csv')
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']
y1= y


reduced_data = training.groupby(training['prognosis']).max()

#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']  
testy    = le.transform(testy)


clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
# print(clf.score(x_train,y_train))
# print ("cross result========")
scores = cross_val_score(clf, x_test, y_test, cv=3)
# print (scores)
print (scores.mean())


model=SVC()
model.fit(x_train,y_train)
print("for svm: ")
print(model.score(x_test,y_test))

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()

    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)

    engine.say(nstr)
    engine.runAndWait()
    engine.stop()


severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index
def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")


def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t",end="->")
    name=input("")
    print("Hello, ",name)

def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]
def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])


def print_disease(node):
    node = node[0]
    val  = node.nonzero() 
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis=",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t",end="->")
        disease_input = input("")
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        if conf==1:
            print("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp=0

            disease_input=cnf_dis[conf_inp]
            break
            # print("Did you mean: ",cnf_dis,"?(yes/no) :",end="")
            # conf_inp = input("")
            # if(conf_inp=="yes"):
            #     break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days=int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter valid input.")
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            # print( "You may have " +  present_disease )
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            # dis_list=list(symptoms_present)
            # if len(dis_list)!=0:
            #     print("symptoms present  " + str(list(symptoms_present)))
            # print("symptoms given "  +  str(list(symptoms_given)) )
            print("Are you experiencing any ")
            symptoms_exp=[]
            for syms in list(symptoms_given):
                inp=""
                print(syms,"? : ",end='')
                while True:
                    inp=input("")
                    if(inp=="yes" or inp=="no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes/no) : ",end="")
                if(inp=="yes"):
                    symptoms_exp.append(syms)

            second_prediction=sec_predict(symptoms_exp)
            # print(second_prediction)
            calc_condition(symptoms_exp,num_days)
            if(present_disease[0]==second_prediction[0]):
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

                # readn(f"You may have {present_disease[0]}")
                # readn(f"{description_list[present_disease[0]]}")

            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            # print(description_list[present_disease[0]])
            precution_list=precautionDictionary[present_disease[0]]
            print("Take following measures : ")
            for  i,j in enumerate(precution_list):
                print(i+1,")",j)

            # confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            # print("confidence level is " + str(confidence_level))

    recurse(0, 1)
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf,cols)
print("----------------------------------------------------------------------------------------")

C:\Users\shafa\AppData\Local\Temp\ipykernel_23476\30028065.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


0.9766058936005257
for svm: 
1.0
-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->

 shafayat


Hello,  shafayat

Enter the symptom you are experiencing  		->

 fever


...

 no


loss_of_balance ? : 

 no


It might not be that bad but you should take precautions.
You may have  Cervical spondylosis or  (vertigo) Paroymsal  Positional Vertigo
Cervical spondylosis is a general term for age-related wear and tear affecting the spinal disks in your neck. As the disks dehydrate and shrink, signs of osteoarthritis develop, including bony projections along the edges of bones (bone spurs).
Benign paroxysmal positional vertigo (BPPV) is one of the most common causes of vertigo â€” the sudden sensation that you're spinning or that the inside of your head is spinning. Benign paroxysmal positional vertigo causes brief episodes of mild to intense dizziness.
Take following measures : 
1 ) use heating pad or cold pack
2 ) exercise
3 ) take otc pain reliver
4 ) consult doctor
----------------------------------------------------------------------------------------


D:\PYTHON INTERPRETER\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [6]:
# Existing code...

# Decision Tree Classifier
clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)

# Cross-Validation Score
scores_dt = cross_val_score(clf, x_test, y_teshast, cv=3)
mean_score_dt = scores_dt.mean()

# Model Accuracy on Test Data
accuracy_dt = clf.score(x_test, y_test)

# Support Vector Machine
model = SVC()
model.fit(x_train, y_train)

# Model Accuracy on Test Data
accuracy_svm = model.score(x_test, y_test)

# Print Model Evaluation
print("Decision Tree Classifier Evaluation:")
print("Cross-Validation Mean Score (DT):", mean_score_dt)
print("Test Accuracy (DT):", accuracy_dt)

print("\nSupport Vector Machine Evaluation:")
print("Test Accuracy (SVM):", accuracy_svm)


Decision Tree Classifier Evaluation:
Cross-Validation Mean Score (DT): 0.9741413218198725
Test Accuracy (DT): 1.0

Support Vector Machine Evaluation:
Test Accuracy (SVM): 1.0


In [7]:
import pandas as pd
# Load the training dataset
training = pd.read_csv('Training.csv')
# Get unique diseases (prognosis) from the dataset
diseases = training['prognosis'].unique()
# Print the list of diseases
print("List of Diseases in the Dataset:")
for disease in diseases:
    print("- " + disease)

List of Diseases in the Dataset:
- Fungal infection
- Allergy
- GERD
- Chronic cholestasis
- Drug Reaction
- Peptic ulcer diseae
- AIDS
- Diabetes 
- Gastroenteritis
- Bronchial Asthma
- Hypertension 
- Migraine
- Cervical spondylosis
- Paralysis (brain hemorrhage)
- Jaundice
- Malaria
- Chicken pox
- Dengue
- Typhoid
- hepatitis A
- Hepatitis B
- Hepatitis C
- Hepatitis D
- Hepatitis E
- Alcoholic hepatitis
- Tuberculosis
- Common Cold
- Pneumonia
- Dimorphic hemmorhoids(piles)
- Heart attack
- Varicose veins
- Hypothyroidism
- Hyperthyroidism
- Hypoglycemia
- Osteoarthristis
- Arthritis
- (vertigo) Paroymsal  Positional Vertigo
- Acne
- Urinary tract infection
- Psoriasis
- Impetigo


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Decision Tree Classifier
train_accuracy = clf.score(x_train, y_train)
cross_val_scores = cross_val_score(clf, x_test, y_test, cv=3)
decision_tree_avg_score = cross_val_scores.mean()

# Support Vector Machine
svm_testing_accuracy = model.score(x_test, y_test)

# Confusion Matrix
y_pred = clf.predict(x_test)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Decision Tree Classifier:")
print("Training Accuracy:", train_accuracy)
print("Cross-Validation Mean Score:", decision_tree_avg_score)
print("Confusion Matrix:")
print(conf_matrix)

print("\nSupport Vector Machine (SVM):")
print("Testing Accuracy:", svm_testing_accuracy)

# Calculate additional scores
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("\nAdditional Scores:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Decision Tree Classifier:
Training Accuracy: 1.0
Cross-Validation Mean Score: 0.9778347691055469
Confusion Matrix:
[[37  0  0 ...  0  0  0]
 [ 0 42  0 ...  0  0  0]
 [ 0  0 42 ...  0  0  0]
 ...
 [ 0  0  0 ... 41  0  0]
 [ 0  0  0 ...  0 40  0]
 [ 0  0  0 ...  0  0 44]]

Support Vector Machine (SVM):
Testing Accuracy: 1.0

Additional Scores:
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [9]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("Testing.csv", delimiter="\t")

# Display the first few rows of the DataFrame
print(df.head())


  itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,diarrhoea,mild_fever,yellow_urine,yellowing_of_eyes,acute_liver_failure,fluid_overload,swelling_of_stomach,swelled_lymph_nodes,malaise,blurred_and_distorted_vision,phlegm,throat_irritation,redness_of_eyes,sinus_pressure,runny_nose,congestion,chest_pain,weakness_in_limbs,fast_heart_rate,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus,neck_pain,dizziness,cramps,bruising,obesity,swollen_legs,swollen_blood_vessels,puffy_face_and_eyes,enlarged_thyr

In [10]:
import pandas as pd

# Load the training dataset
training_data = pd.read_csv('Training.csv')

# Check the columns in training_data
print("Columns in training_data:", training_data.columns)


Columns in training_data: Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue',
       ...
       'blackheads', 'scurring', 'skin_peeling', 'silver_like_dusting',
       'small_dents_in_nails', 'inflammatory_nails', 'blister',
       'red_sore_around_nose', 'yellow_crust_ooze', 'prognosis'],
      dtype='object', length=133)


In [15]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import csv
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns
cols = cols[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)

scores = cross_val_score(clf, x_test, y_test, cv=3)
print("Decision Tree Scores:", scores)
print("Mean Accuracy:", scores.mean())

model = SVC()
model.fit(x_train, y_train)
svm_score = model.score(x_test, y_test)
print("SVM Score:", svm_score)



Decision Tree Scores: [0.97601476 0.97227357 0.97966728]
Mean Accuracy: 0.9759852034749552
SVM Score: 1.0


In [16]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
from sklearn.metrics import confusion_matrix, accuracy_score

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns
cols = cols[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)

scores = cross_val_score(clf, x_test, y_test, cv=3)
print("Decision Tree Cross Validation Scores:", scores)
print("Decision Tree Mean Accuracy:", scores.mean())

model = SVC()
model.fit(x_train, y_train)
svm_score = model.score(x_test, y_test)
print("SVM Score:", svm_score)

y_pred = clf.predict(x_test)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
    symptoms_dict[symptom] = index

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum += severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("It might not be that bad but you should take precautions.")

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description = {row[0]: row[1]}
            description_list.update(_description)

def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass

def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
            precautionDictionary.update(_prec)

def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello, ", name)

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing  \t\t", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Searches related to input: ")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days = int(input("Okay. From how many days? : "))
            break
        except:
            print("Enter valid input.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of the following symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp == "yes" or inp == "no":
                        break
                    else:
                        print("Provide proper answers i.e. (yes/no) : ", end="")
                if inp == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            precution_list = precautionDictionary[present_disease[0]]
            print("Take the following measures:")
            for i, j in enumerate(precution_list):
                print(i + 1, ")", j)

    recurse(0, 1)

getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf, cols)
print("----------------------------------------------------------------------------------------")


Decision Tree Cross Validation Scores: [0.97232472 0.98151571 0.97781885]
Decision Tree Mean Accuracy: 0.9772197629554854
SVM Score: 1.0
Confusion Matrix:
[[37  0  0 ...  0  0  0]
 [ 0 42  0 ...  0  0  0]
 [ 0  0 42 ...  0  0  0]
 ...
 [ 0  0  0 ... 41  0  0]
 [ 0  0  0 ...  0 40  0]
 [ 0  0  0 ...  0  0 44]]
-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->

 bhanu


Hello,  bhanu

Enter the symptom you are experiencing  		->

 fever


Searches related to input: 
0 ) high_fever
1 ) mild_fever
Select the one you meant (0 - 1):  

...

 no


belly_pain ? : 

 no


It might not be that bad but you should take precautions.
You may have  Typhoid or  Peptic ulcer diseae
An acute illness characterized by fever caused by infection with the bacterium Salmonella typhi. Typhoid fever has an insidious onset, with fever, headache, constipation, malaise, chills, and muscle pain. Diarrhea is uncommon, and vomiting is not usually severe.
Peptic ulcer disease (PUD) is a break in the inner lining of the stomach, the first part of the small intestine, or sometimes the lower esophagus. An ulcer in the stomach is called a gastric ulcer, while one in the first part of the intestines is a duodenal ulcer.
Take the following measures:
1 ) eat high calorie vegitables
2 ) antiboitic therapy
3 ) consult doctor
4 ) medication
----------------------------------------------------------------------------------------


D:\PYTHON INTERPRETER\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [17]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

# Load the training data
training = pd.read_csv('Training.csv')

# Get feature columns and target column
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']

# Encoding target labels
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Train a Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the full confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[37  0  0 ...  0  0  0]
 [ 0 42  0 ...  0  0  0]
 [ 0  0 42 ...  0  0  0]
 ...
 [ 0  0  0 ... 41  0  0]
 [ 0  0  0 ...  0 40  0]
 [ 0  0  0 ...  0  0 44]]


In [18]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

# Load the training data
training = pd.read_csv('Training.csv')

# Get feature columns and target column
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']

# Encoding target labels
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Train a Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(x_test)

# Calculate precision, recall, F1-score, and support
report = classification_report(y_test, y_pred, target_names=le.classes_)

# Print the classification report
print("Classification Report:")
print(report)


Classification Report:
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        37
                                   AIDS       1.00      1.00      1.00        42
                                   Acne       1.00      1.00      1.00        42
                    Alcoholic hepatitis       1.00      1.00      1.00        40
                                Allergy       1.00      1.00      1.00        36
                              Arthritis       1.00      1.00      1.00        42
                       Bronchial Asthma       1.00      1.00      1.00        48
                   Cervical spondylosis       1.00      1.00      1.00        37
                            Chicken pox       1.00      1.00      1.00        38
                    Chronic cholestasis       1.00      1.00      1.00        31
                            Common Cold       1.00      1.00      1.00        34
    

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the dataset
url = "https://raw.githubusercontent.com/matheusvpt/healthcare-datasets/main/symptoms_diseases/Training.csv"
df = pd.read_csv("Training.csv")

# Display the first few rows of the dataframe
print("Sample of the dataset:")
print(df.head())

# Split the data into features (X) and target (y)
X = df.drop(columns=['prognosis'])
y = df['prognosis']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Sample of the dataset:
   itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  \
0       0           0             0        0                 0  ...   
1       0           0             0        0                 0  ...   
2       0           0             0        0                 0  ...   
3       0           0             0        0                 0  ...   
4       0           0             0        0                 0  ...   

   blackheads  scurring  skin_peeling  silver_like_dusting  \
0           0

In [20]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns
cols = cols[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf1 = DecisionTreeClassifier()
clf = clf1.fit(x_train, y_train)
scores = cross_val_score(clf, x_test, y_test, cv=3)
print("Decision Tree Classifier Accuracy:", scores.mean())

model = SVC()
model.fit(x_train, y_train)
print("SVM Classifier Accuracy:", model.score(x_test, y_test))

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {symptom: index for index, symptom in enumerate(x)}

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum += severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take consultation from a doctor.")
    else:
        print("It might not be that bad but you should take precautions.")

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            description_list[row[0]] = row[1]

def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) == 2:
                severityDictionary[row[0]] = int(row[1])

def getPrecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            precautionDictionary[row[0]] = [row[1], row[2], row[3], row[4]]

def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello,", name)

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing\t\t", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Searches related to input:")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter a valid symptom.")

    while True:
        try:
            num_days = int(input("From how many days? : "))
            break
        except ValueError:
            print("Enter a valid number of days.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of the following?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp.lower() == "yes" or inp.lower() == "no":
                        break
                    else:
                        print("Please provide valid answers (yes/no) : ", end="")
                if inp.lower() == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have:", present_disease[0])
                print("Description:", description_list[present_disease[0]])
            else:
                print("You may have:", present_disease[0], "or", second_prediction[0])
                print("Description for", present_disease[0], ":", description_list[present_disease[0]])
                print("Description for", second_prediction[0], ":", description_list[second_prediction[0]])

            print("Take the following precautions:")
            precution_list = precautionDictionary[present_disease[0]]
            for i, j in enumerate(precution_list):
                print(i + 1, ")", j)

    recurse(0, 1)

getSeverityDict()
getDescription()
getPrecautionDict()
getInfo()
tree_to_code(clf, cols)
print("----------------------------------------------------------------------------------------")


Decision Tree Classifier Accuracy: 0.9722906193941792
SVM Classifier Accuracy: 1.0
-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->

 Sg


Hello, Sg

Enter the symptom you are experiencing		->

 fever


Searches related to input:
0 ) high_fever
1 ) mild_fever
Select the one you meant (0 - 1):  

...

 no


red_spots_over_body ? : 

 no


It might not be that bad but you should take precautions.
You may have: Chicken pox or Impetigo
Description for Chicken pox : Chickenpox is a highly contagious disease caused by the varicella-zoster virus (VZV). It can cause an itchy, blister-like rash. The rash first appears on the chest, back, and face, and then spreads over the entire body, causing between 250 and 500 itchy blisters.
Description for Impetigo : Impetigo (im-puh-TIE-go) is a common and highly contagious skin infection that mainly affects infants and children. Impetigo usually appears as red sores on the face, especially around a child's nose and mouth, and on hands and feet. The sores burst and develop honey-colored crusts.
Take the following precautions:
1 ) use neem in bathing 
2 ) consume neem leaves
3 ) take vaccine
4 ) avoid public places
----------------------------------------------------------------------------------------


D:\PYTHON INTERPRETER\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [21]:
pip install openai

In [ ]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def get_hospital_for_disease_district(district, disease):
    hospital_data = {
        "Guntur": {
            "Fungal infection": "Ramesh Hospitals",
            "Allergy": "Sri Venkateswara Hospitals",
            "GERD": "Guntur Medical College Hospital",
            "Chronic cholestasis": "Sri Raghava Hospital",
            "Drug Reaction": "Guntur General Hospital",
            "Peptic ulcer disease": "Sri Raghava Hospital",
            "AIDS": "Andhra Hospitals",
            "Diabetes": "Sri Ram Hospitals",  # Updated hospital for Diabetes
            "Gastroenteritis": "Sri Raghava Hospital",
            "Bronchial Asthma": "Guntur Medical College Hospital",
            "Hypertension": "Sri Raghava Hospital",
            "Migraine": "Guntur Medical College Hospital",
            "Cervical spondylosis": "Sri Raghava Hospital",
            "Paralysis (brain hemorrhage)": "Sri Raghava Hospital",
            "Jaundice": "Guntur General Hospital",
            "Malaria": "Sri Venkateswara Hospitals",
            "Chickenpox": "Guntur Medical College Hospital",
            "Dengue": "Andhra Hospitals",
            "Typhoid": "Andhra Hospitals",
            "Hepatitis A": "Andhra Hospitals",
            "Hepatitis B": "Andhra Hospitals",
            "Hepatitis C": "Andhra Hospitals",
            "Hepatitis D": "Andhra Hospitals",
            "Hepatitis E": "Andhra Hospitals",
            "Alcoholic hepatitis": "Sri Raghava Hospital",
            "Tuberculosis": "Guntur Medical College Hospital",
            "Common Cold": "Sri Venkateswara Hospitals",
            "Pneumonia": "Guntur General Hospital",
            "Dimorphic hemmorhoids (piles)": "Sri Raghava Hospital",
            "Heart attack": "Guntur Medical College Hospital",
            "Varicose veins": "Sri Venkateswara Hospitals",
            "Hypothyroidism": "Guntur General Hospital",
            "Hyperthyroidism": "Sri Venkateswara Hospitals",
            "Hypoglycemia": "Sri Venkateswara Hospitals",
            "Osteoarthritis": "Sri Raghava Hospital",
            "Arthritis": "Sri Raghava Hospital",
            "(Vertigo) Paroxysmal Positional Vertigo": "Guntur Medical College Hospital",
            "Acne": "Sri Venkateswara Hospitals",
            "Urinary tract infection": "Sri Venkateswara Hospitals",
            "Psoriasis": "Sri Venkateswara Hospitals",
            "Impetigo": "Guntur Medical College Hospital"
        },
        "Krishna": {
            # Define hospitals for diseases in Krishna district here
        },
        "East Godavari": {
            # Define hospitals for diseases in East Godavari district here
        },
        # Add more districts as needed
    }

    district = district.capitalize()

    if district in hospital_data:
        if disease in hospital_data[district]:
            return hospital_data[district][disease]
        else:
            return "Hospital not found for the given disease in the specified district."
    else:
        return "District not found."


def get_hospital_for_user_input():
    district = input("Enter your district: ")
    disease = input("Enter the disease you are experiencing: ")
    hospital = get_hospital_for_disease_district(district, disease)
    print(f"The hospital for {disease} in {district} district is: {hospital}")


def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description = {row[0]: row[1]}
            description_list.update(_description)


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
            precautionDictionary.update(_prec)


def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello, ", name)


def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if(len(pred_list) > 0):
        return 1, pred_list
    else:
        return 0, []


def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])


def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))


def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("searches related to input: ")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
            # print("Did you mean: ",cnf_dis,"?(yes/no) :",end="")
            # conf_inp = input("")
            # if(conf_inp=="yes"):
            #     break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days = int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter valid input.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            # print( "You may have " +  present_disease )
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            # dis_list=list(symptoms_present)
            # if len(dis_list)!=0:
            #     print("symptoms present  " + str(list(symptoms_present)))
            # print("symptoms given "  +  str(list(symptoms_given)) )
            print("Are you experiencing any ")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if(inp == "yes" or inp == "no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes/no) : ", end="")
                if(inp == "yes"):
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            # print(second_prediction)
            calc_condition(symptoms_exp, num_days)
            if(present_disease[0] == second_prediction[0]):
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

                # readn(f"You may have {present_disease[0]}")
                # readn(f"{description_list[present_disease[0]]}")

            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            # print(description_list[present_disease[0]])
            precution_list = precautionDictionary[present_disease[0]]
            print("Take following measures : ")
            for i, j in enumerate(precution_list):
                print(i+1, ")", j)

            # confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            # print("confidence level is " + str(confidence_level))

    recurse(0, 1)


def calc_condition(exp, days):
    sum = 0
    for item in exp:
         sum = sum + severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("It might not be that bad but you should take precautions.")


severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index


def main():
    # Load data and models
    training = pd.read_csv('Training.csv')
    cols = training.columns[:-1]
    x = training[cols]
    y = training['prognosis']
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

    clf = DecisionTreeClassifier()
    clf.fit(x_train, y_train)

    model = SVC()
    model.fit(x_train, y_train)

    reduced_data = training.groupby(training['prognosis']).max()

    # Get data for prediction
    getDescription()
    getSeverityDict()
    getprecautionDict()

    getInfo()

    # Get disease prediction
    tree_to_code(clf, cols)
    print("----------------------------------------------------------------------------------------")

    # Get hospital for disease in district
    get_hospital_for_user_input()


if __name__ == "__main__":
    main()


In [ ]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import train_test_split
import csv

# Hospital data for Andhra Pradesh
hospital_data = {
    "Guntur": {
        "Fungal infection": "Ramesh Hospitals",
        "Allergy": "Sri Venkateswara Hospitals",
        "GERD": "Guntur Medical College Hospital",
        "Chronic cholestasis": "Sri Raghava Hospital",
        "Drug Reaction": "Guntur General Hospital",
        "Peptic ulcer disease": "Sri Raghava Hospital",
        "AIDS": "Andhra Hospitals",
        "Diabetes": "Sri Ram Hospitals",  # Updated hospital for Diabetes
        "Gastroenteritis": "Sri Raghava Hospital",
        "Bronchial Asthma": "Guntur Medical College Hospital",
        "Hypertension": "Sri Raghava Hospital",
        "Migraine": "Guntur Medical College Hospital",
        "Cervical spondylosis": "Sri Raghava Hospital",
        "Paralysis (brain hemorrhage)": "Sri Raghava Hospital",
        "Jaundice": "Guntur General Hospital",
        "Malaria": "Sri Venkateswara Hospitals",
        "Chickenpox": "Guntur Medical College Hospital",
        "Dengue": "Andhra Hospitals",
        "Typhoid": "Andhra Hospitals",
        "Hepatitis A": "Andhra Hospitals",
        "Hepatitis B": "Andhra Hospitals",
        "Hepatitis C": "Andhra Hospitals",
        "Hepatitis D": "Andhra Hospitals",
        "Hepatitis E": "Andhra Hospitals",
        "Alcoholic hepatitis": "Sri Raghava Hospital",
        "Tuberculosis": "Guntur Medical College Hospital",
        "Common Cold": "Sri Venkateswara Hospitals",
        "Pneumonia": "Guntur General Hospital",
        "Dimorphic hemmorhoids (piles)": "Sri Raghava Hospital",
        "Heart attack": "Guntur Medical College Hospital",
        "Varicose veins": "Sri Venkateswara Hospitals",
        "Hypothyroidism": "Guntur General Hospital",
        "Hyperthyroidism": "Sri Venkateswara Hospitals",
        "Hypoglycemia": "Sri Venkateswara Hospitals",
        "Osteoarthritis": "Sri Raghava Hospital",
        "Arthritis": "Sri Raghava Hospital",
        "(Vertigo) Paroxysmal Positional Vertigo": "Guntur Medical College Hospital",
        "Acne": "Sri Venkateswara Hospitals",
        "Urinary tract infection": "Sri Venkateswara Hospitals",
        "Psoriasis": "Sri Venkateswara Hospitals",
        "Impetigo": "Guntur Medical College Hospital"
    },
    # Add more districts with hospitals for diseases as needed
}

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

model = SVC()
model.fit(x_train, y_train)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
    symptoms_dict[symptom] = index

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum += severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("It might not be that bad, but you should take precautions.")

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row:  # Check if the row is not empty
                _description = {row[0]: row[1]}
                description_list.update(_description)

def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 2:  # Check if row is not empty and has two columns
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)

def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 5:  # Check if row is not empty and has five columns
                _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
                precautionDictionary.update(_prec)

def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello,", name)

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing: ", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Searches related to input: ")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}): ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter a valid symptom.")

    while True:
        try:
            num_days = int(input("From how many days are you experiencing this? : "))
            break
        except ValueError:
            print("Please enter a valid number of days.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of these symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp.lower() == "yes" or inp.lower() == "no":
                        break
                    else:
                        print("Please provide proper answers i.e. (yes/no) : ", end="")
                if inp.lower() == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

            else:
                print("You may have ", present_disease[0], "or", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            print("Take the following measures : ")
            precaution_list = precautionDictionary[present_disease[0]]
            for i, j in enumerate(precaution_list):
                print(i + 1, ")", j)

            district = input("Enter your district: ")
            hospital_name = hospital_data.get(district, {}).get(present_disease[0])
            if hospital_name:
                print("Recommended Hospital:", hospital_name)
            else:
                print("No hospital found for the given district and disease.")

    recurse(0, 1)

def main():
    getDescription()
    getSeverityDict()
    getprecautionDict()
    getInfo()
    tree_to_code(clf, cols)
    print("----------------------------------------------------------------------------------------")

if __name__ == "__main__":
    main()


In [ ]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import train_test_split
import csv

# Hospital data for Andhra Pradesh
hospital_data = {
    "Guntur": {
        "Fungal infection": "Ramesh Hospitals",
        "Allergy": "Sri Venkateswara Hospitals",
        "GERD": "Guntur Medical College Hospital",
        "Chronic cholestasis": "Sri Raghava Hospital",
        "Drug Reaction": "Guntur General Hospital",
        "Peptic ulcer disease": "Sri Raghava Hospital",
        "AIDS": "Andhra Hospitals",
        "Diabetes": "Sri Ram Hospitals",  # Updated hospital for Diabetes
        "Gastroenteritis": "Sri Raghava Hospital",
        "Bronchial Asthma": "Guntur Medical College Hospital",
        "Hypertension": "Sri Raghava Hospital",
        "Migraine": "Guntur Medical College Hospital",
        "Cervical spondylosis": "Sri Raghava Hospital",
        "Paralysis (brain hemorrhage)": "Sri Raghava Hospital",
        "Jaundice": "Guntur General Hospital",
        "Malaria": "Sri Venkateswara Hospitals",
        "Chickenpox": "Guntur Medical College Hospital",
        "Dengue": "Andhra Hospitals",
        "Typhoid": "Andhra Hospitals",
        "Hepatitis A": "Andhra Hospitals",
        "Hepatitis B": "Andhra Hospitals",
        "Hepatitis C": "Andhra Hospitals",
        "Hepatitis D": "Andhra Hospitals",
        "Hepatitis E": "Andhra Hospitals",
        "Alcoholic hepatitis": "Sri Raghava Hospital",
        "Tuberculosis": "Guntur Medical College Hospital",
        "Common Cold": "Sri Venkateswara Hospitals",
        "Pneumonia": "Guntur General Hospital",
        "Dimorphic hemmorhoids (piles)": "Sri Raghava Hospital",
        "Heart attack": "Guntur Medical College Hospital",
        "Varicose veins": "Sri Venkateswara Hospitals",
        "Hypothyroidism": "Guntur General Hospital",
        "Hyperthyroidism": "Sri Venkateswara Hospitals",
        "Hypoglycemia": "Sri Venkateswara Hospitals",
        "Osteoarthritis": "Sri Raghava Hospital",
        "Arthritis": "Sri Raghava Hospital",
        "(Vertigo) Paroxysmal Positional Vertigo": "Guntur Medical College Hospital",
        "Acne": "Sri Venkateswara Hospitals",
        "Urinary tract infection": "Sri Venkateswara Hospitals",
        "Psoriasis": "Sri Venkateswara Hospitals",
        "Impetigo": "Guntur Medical College Hospital"
    },
    # Add more districts with hospitals for diseases as needed
}

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

model = SVC()
model.fit(x_train, y_train)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
    symptoms_dict[symptom] = index

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum += severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("It might not be that bad, but you should take precautions.")

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row:  # Check if the row is not empty
                _description = {row[0]: row[1]}
                description_list.update(_description)

def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 2:  # Check if row is not empty and has two columns
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)

def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 5:  # Check if row is not empty and has five columns
                _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
                precautionDictionary.update(_prec)

def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello,", name)

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing: ", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Searches related to input: ")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}): ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter a valid symptom.")

    while True:
        try:
            num_days = int(input("From how many days are you experiencing this? : "))
            break
        except ValueError:
            print("Please enter a valid number of days.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of these symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp.lower() == "yes" or inp.lower() == "no":
                        break
                    else:
                        print("Please provide proper answers i.e. (yes/no) : ", end="")
                if inp.lower() == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

            else:
                print("You may have ", present_disease[0], "or", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            print("Take the following measures : ")
            precaution_list = precautionDictionary[present_disease[0]]
            for i, j in enumerate(precaution_list):
                print(i + 1, ")", j)

            district = input("Enter your district: ")
            hospital_name = hospital_data.get(district, {}).get(present_disease[0])
            if hospital_name:
                print("Recommended Hospital:", hospital_name)
            else:
                print("No hospital found for the given district and disease.")

    recurse(0, 1)

def main():
    getDescription()
    getSeverityDict()
    getprecautionDict()
    getInfo()
    tree_to_code(clf, cols)
    print("----------------------------------------------------------------------------------------")

if __name__ == "__main__":
    main()


In [7]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  # Add this import
import numpy as np
import csv

# Hospital data for Andhra Pradesh
hospital_data = {
    "Guntur": {
        "Fungal infection": "Ramesh Hospitals",
        "Allergy": "Sri Venkateswara Hospitals",
        "GERD": "Guntur Medical College Hospital",
        "Chronic cholestasis": "Sri Raghava Hospital",
        "Drug Reaction": "Guntur General Hospital",
        "Peptic ulcer disease": "Sri Raghava Hospital",
        "AIDS": "Andhra Hospitals",
        "Diabetes": "Sri Ram Hospitals",  # Updated hospital for Diabetes
        "Gastroenteritis": "Sri Raghava Hospital",
        "Bronchial Asthma": "Guntur Medical College Hospital",
        "Hypertension": "Sri Raghava Hospital",
        "Migraine": "Guntur Medical College Hospital",
        "Cervical spondylosis": "Sri Raghava Hospital",
        "Paralysis (brain hemorrhage)": "Sri Raghava Hospital",
        "Jaundice": "Guntur General Hospital",
        "Malaria": "Sri Venkateswara Hospitals",
        "Chickenpox": "Guntur Medical College Hospital",
        "Dengue": "Andhra Hospitals",
        "Typhoid": "Andhra Hospitals",
        "Hepatitis A": "Andhra Hospitals",
        "Hepatitis B": "Andhra Hospitals",
        "Hepatitis C": "Andhra Hospitals",
        "Hepatitis D": "Andhra Hospitals",
        "Hepatitis E": "Andhra Hospitals",
        "Alcoholic hepatitis": "Sri Raghava Hospital",
        "Tuberculosis": "Guntur Medical College Hospital",
        "Common Cold": "Sri Venkateswara Hospitals",
        "Pneumonia": "Guntur General Hospital",
        "Dimorphic hemmorhoids (piles)": "Sri Raghava Hospital",
        "Heart attack": "Guntur Medical College Hospital",
        "Varicose veins": "Sri Venkateswara Hospitals",
        "Hypothyroidism": "Guntur General Hospital",
        "Hyperthyroidism": "Sri Venkateswara Hospitals",
        "Hypoglycemia": "Sri Venkateswara Hospitals",
        "Osteoarthritis": "Sri Raghava Hospital",
        "Arthritis": "Sri Raghava Hospital",
        "(Vertigo) Paroxysmal Positional Vertigo": "Guntur Medical College Hospital",
        "Acne": "Sri Venkateswara Hospitals",
        "Urinary tract infection": "Sri Venkateswara Hospitals",
        "Psoriasis": "Sri Venkateswara Hospitals",
        "Impetigo": "Guntur Medical College Hospital"
    },
    # Add more districts with hospitals for diseases as needed
}

training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

reduced_data = training.groupby(training['prognosis']).max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = testing['prognosis']
testy = le.transform(testy)

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

model = SVC()  # Fixed the import here
model.fit(x_train, y_train)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

severityDictionary = dict()
description_list = dict()
precautionDictionary = dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
    symptoms_dict[symptom] = index

def calc_condition(exp, days):
    sum = 0
    for item in exp:
        sum += severityDictionary[item]
    if (sum * days) / (len(exp) + 1) > 13:
        print("You should take the consultation from a doctor.")
    else:
        print("It might not be that bad, but you should take precautions.")

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row:  # Check if the row is not empty
                _description = {row[0]: row[1]}
                description_list.update(_description)

def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 2:  # Check if row is not empty and has two columns
                _diction = {row[0]: int(row[1])}
                severityDictionary.update(_diction)

def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row and len(row) == 5:  # Check if row is not empty and has five columns
                _prec = {row[0]: [row[1], row[2], row[3], row[4]]}
                precautionDictionary.update(_prec)

def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t", end="->")
    name = input("")
    print("Hello,", name)

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []

    while True:
        print("\nEnter the symptom you are experiencing: ", end="->")
        disease_input = input("")
        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("Searches related to input: ")
            for num, it in enumerate(cnf_dis):
                print(num, ")", it)
            if num != 0:
                print(f"Select the one you meant (0 - {num}): ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter a valid symptom.")

    while True:
        try:
            num_days = int(input("From how many days are you experiencing this? : "))
            break
        except ValueError:
            print("Please enter a valid number of days.")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("Are you experiencing any of these symptoms?")
            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                print(syms, "? : ", end='')
                while True:
                    inp = input("")
                    if inp.lower() == "yes" or inp.lower() == "no":
                        break
                    else:
                        print("Please provide proper answers i.e. (yes/no) : ", end="")
                if inp.lower() == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                print("You may have ", present_disease[0])
                print(description_list[present_disease[0]])

            else:
                print("You may have ", present_disease[0], "or", second_prediction[0])
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])

            print("Take the following measures : ")
            precaution_list = precautionDictionary[present_disease[0]]
            for i, j in enumerate(precaution_list):
                print(i + 1, ")", j)

            district = input("Enter your district: ")
            hospital_name = hospital_data.get(district, {}).get(present_disease[0])
            if hospital_name:
                print("Recommended Hospital:", hospital_name)
            else:
                print("No hospital found for the given district and disease.")

    recurse(0, 1)

def main():
    getDescription()
    getSeverityDict()
    getprecautionDict()
    getInfo()
    tree_to_code(clf, cols)
    print("----------------------------------------------------------------------------------------")

if __name__ == "__main__":
    main()


NameError: name 'predicted_labels' is not defined

Symptom Severity:
skin_rash: 3
nodal_skin_eruptions: 4
continuous_sneezing: 4
shivering: 5
chills: 3
joint_pain: 3
stomach_pain: 5
acidity: 3
ulcers_on_tongue: 4
muscle_wasting: 3
vomiting: 5
burning_micturition: 6
spotting_urination: 6
fatigue: 4
weight_gain: 3
anxiety: 4
cold_hands_and_feets: 5
mood_swings: 3
weight_loss: 3
restlessness: 5
lethargy: 2
patches_in_throat: 6
irregular_sugar_level: 5
cough: 4
high_fever: 7
sunken_eyes: 3
breathlessness: 4
sweating: 3
dehydration: 4
indigestion: 5
headache: 3
yellowish_skin: 3
dark_urine: 4
nausea: 5
loss_of_appetite: 4
pain_behind_the_eyes: 4
back_pain: 3
constipation: 4
abdominal_pain: 4
diarrhoea: 6
mild_fever: 5
yellow_urine: 4
yellowing_of_eyes: 4
acute_liver_failure: 6
fluid_overload: 6
swelling_of_stomach: 7
swelled_lymph_nodes: 6
malaise: 6
blurred_and_distorted_vision: 5
phlegm: 5
throat_irritation: 4
redness_of_eyes: 5
sinus_pressure: 4
runny_nose: 5
congestion: 5
chest_pain: 7
weakness_in_limbs: 7
fast_heart_rate: 5
pain_during

IndexError: list index out of range

In [2]:
import csv

# Path to the symptom_Description.csv file
file_path = 'symptom_Description.csv'

def print_symptom_description():
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        print("Symptom Descriptions:")
        for row in csv_reader:
            symptom = row[0]
            description = row[1]
            print(f"{symptom}: {description}")

# Call the function to print the contents of symptom_Description.csv
print_symptom_description()


Symptom Descriptions:
Malaria: An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.
Allergy: An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens.
Hypothyroidism: Hypothyroidism, also called underactive thyroid or low thyroid, is a disorder of the endocrine system in which the thyroid gland does not produce enough thyroid hormone.
Psoriasis: Psoriasis is a common skin disorder that forms thick, red, bumpy patches covered with silvery scales. They can pop up anywhere, but most appear on the scalp, elbows, knees, and lower back. Psoriasis can't be passed from person to person. It does sometimes happen in members of the same family.
G

In [3]:
import csv

# Path to the symptom_precaution.csv file
file_path = 'symptom_precaution.csv'

def print_symptom_precaution():
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        print("Symptom Precautions:")
        for row in csv_reader:
            symptom = row[0]
            precaution1 = row[1]
            precaution2 = row[2]
            precaution3 = row[3]
            precaution4 = row[4]
            print(f"{symptom}:")
            print(f"1. {precaution1}")
            print(f"2. {precaution2}")
            print(f"3. {precaution3}")
            print(f"4. {precaution4}")
            print()

# Call the function to print the contents of symptom_precaution.csv
print_symptom_precaution()


Symptom Precautions:
Malaria:
1. Consult nearest hospital
2. avoid oily food
3. avoid non veg food
4. keep mosquitos out

Allergy:
1. apply calamine
2. cover area with bandage
3. 
4. use ice to compress itching

Hypothyroidism:
1. reduce stress
2. exercise
3. eat healthy
4. get proper sleep

Psoriasis:
1. wash hands with warm soapy water
2. stop bleeding using pressure
3. consult doctor
4. salt baths

GERD:
1. avoid fatty spicy food
2. avoid lying down after eating
3. maintain healthy weight
4. exercise

Chronic cholestasis:
1. cold baths
2. anti itch medicine
3. consult doctor
4. eat healthy

hepatitis A:
1. Consult nearest hospital
2. wash hands through
3. avoid fatty spicy food
4. medication

Osteoarthristis:
1. acetaminophen
2. consult nearest hospital
3. follow up
4. salt baths

(vertigo) Paroymsal  Positional Vertigo:
1. lie down
2. avoid sudden change in body
3. avoid abrupt head movment
4. relax

Hypoglycemia:
1. lie down on side
2. check in pulse
3. drink sugary drinks
4. cons

In [4]:
# Hospital data for Guntur district
hospital_data_guntur = {
    "Fungal infection": "Ramesh Hospitals",
    "Allergy": "Sri Venkateswara Hospitals",
    "GERD": "Guntur Medical College Hospital",
    "Chronic cholestasis": "Sri Raghava Hospital",
    "Drug Reaction": "Guntur General Hospital",
    "Peptic ulcer disease": "Sri Raghava Hospital",
    "AIDS": "Andhra Hospitals",
    "Diabetes": "Sri Ram Hospitals",
    "Gastroenteritis": "Sri Raghava Hospital",
    "Bronchial Asthma": "Guntur Medical College Hospital",
    "Hypertension": "Sri Raghava Hospital",
    "Migraine": "Guntur Medical College Hospital",
    "Cervical spondylosis": "Sri Raghava Hospital",
    "Paralysis (brain hemorrhage)": "Sri Raghava Hospital",
    "Jaundice": "Guntur General Hospital",
    "Malaria": "Sri Venkateswara Hospitals",
    "Chickenpox": "Guntur Medical College Hospital",
    "Dengue": "Andhra Hospitals",
    "Typhoid": "Andhra Hospitals",
    "Hepatitis A": "Andhra Hospitals",
    "Hepatitis B": "Andhra Hospitals",
    "Hepatitis C": "Andhra Hospitals",
    "Hepatitis D": "Andhra Hospitals",
    "Hepatitis E": "Andhra Hospitals",
    "Alcoholic hepatitis": "Sri Raghava Hospital",
    "Tuberculosis": "Guntur Medical College Hospital",
    "Common Cold": "Sri Venkateswara Hospitals",
    "Pneumonia": "Guntur General Hospital",
    "Dimorphic hemmorhoids (piles)": "Sri Raghava Hospital",
    "Heart attack": "Guntur Medical College Hospital",
    "Varicose veins": "Sri Venkateswara Hospitals",
    "Hypothyroidism": "Guntur General Hospital",
    "Hyperthyroidism": "Sri Venkateswara Hospitals",
    "Hypoglycemia": "Sri Venkateswara Hospitals",
    "Osteoarthritis": "Sri Raghava Hospital",
    "Arthritis": "Sri Raghava Hospital",
    "(Vertigo) Paroxysmal Positional Vertigo": "Guntur Medical College Hospital",
    "Acne": "Sri Venkateswara Hospitals",
    "Urinary tract infection": "Sri Venkateswara Hospitals",
    "Psoriasis": "Sri Venkateswara Hospitals",
    "Impetigo": "Guntur Medical College Hospital"
}

def print_guntur_hospitals():
    print("Hospitals in Guntur District:")
    for disease, hospital in hospital_data_guntur.items():
        print(f"{disease}: {hospital}")

# Call the function to print the hospitals dataset for Guntur district
print_guntur_hospitals()


Hospitals in Guntur District:
Fungal infection: Ramesh Hospitals
Allergy: Sri Venkateswara Hospitals
GERD: Guntur Medical College Hospital
Chronic cholestasis: Sri Raghava Hospital
Drug Reaction: Guntur General Hospital
Peptic ulcer disease: Sri Raghava Hospital
AIDS: Andhra Hospitals
Diabetes: Sri Ram Hospitals
Gastroenteritis: Sri Raghava Hospital
Bronchial Asthma: Guntur Medical College Hospital
Hypertension: Sri Raghava Hospital
Migraine: Guntur Medical College Hospital
Cervical spondylosis: Sri Raghava Hospital
Paralysis (brain hemorrhage): Sri Raghava Hospital
Jaundice: Guntur General Hospital
Malaria: Sri Venkateswara Hospitals
Chickenpox: Guntur Medical College Hospital
Dengue: Andhra Hospitals
Typhoid: Andhra Hospitals
Hepatitis A: Andhra Hospitals
Hepatitis B: Andhra Hospitals
Hepatitis C: Andhra Hospitals
Hepatitis D: Andhra Hospitals
Hepatitis E: Andhra Hospitals
Alcoholic hepatitis: Sri Raghava Hospital
Tuberculosis: Guntur Medical College Hospital
Common Cold: Sri Venkate

In [5]:
def provide_results(predicted_disease):
    print("----------------------------------------------------------------------------------------")
    print("RESULTS:")
    print("----------------------------------------------------------------------------------------")
    print("Based on the symptoms provided, the predicted disease is:")
    print(predicted_disease)
    print("----------------------------------------------------------------------------------------")
    print("DESCRIPTION:")
    print(description_list[predicted_disease])
    print("----------------------------------------------------------------------------------------")
    print("PRECAUTIONS:")
    precautions = precautionDictionary[predicted_disease]
    for index, precaution in enumerate(precautions, 1):
        print(f"{index}. {precaution}")
    print("----------------------------------------------------------------------------------------")
    print("RECOMMENDED HOSPITAL:")
    district = input("Enter your district (e.g., Guntur, Krishna, etc.): ").strip().title()
    if district in hospital_data:
        if predicted_disease in hospital_data[district]:
            print(f"Hospital: {hospital_data[district][predicted_disease]}")
        else:
            print("No specific hospital found for the predicted disease in this district.")
    else:
        print("District not found in the hospital data.")
    print("----------------------------------------------------------------------------------------")

# Assuming you have the predicted_disease from your model
predicted_disease = "Malaria"  # Example predicted disease, replace with actual prediction
provide_results(predicted_disease)


----------------------------------------------------------------------------------------
RESULTS:
----------------------------------------------------------------------------------------
Based on the symptoms provided, the predicted disease is:
Malaria
----------------------------------------------------------------------------------------
DESCRIPTION:


NameError: name 'description_list' is not defined

In [9]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Load the datasets
training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')

# Prepare data
cols = training.columns
cols = cols[:-1]
x = training[cols]
y = training['prognosis']
y1 = y

# Label Encoding
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Train the Decision Tree Classifier
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

# Predict on the test data
y_pred = clf.predict(x_test)

# Calculate performance metrics
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the results
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)
print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Confusion Matrix:
[[37  0  0 ...  0  0  0]
 [ 0 42  0 ...  0  0  0]
 [ 0  0 42 ...  0  0  0]
 ...
 [ 0  0  0 ... 41  0  0]
 [ 0  0  0 ...  0 40  0]
 [ 0  0  0 ...  0  0 44]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        37
           1       1.00      1.00      1.00        42
           2       1.00      1.00      1.00        42
           3       1.00      1.00      1.00        40
           4       1.00      1.00      1.00        36
           5       1.00      1.00      1.00        42
           6       1.00      1.00      1.00        48
           7       1.00      1.00      1.00        37
           8       1.00      1.00      1.00        38
           9       1.00      1.00      1.00        31
          10       1.00      1.00      1.00        34
          11       1.00      1.00      1.00        46
          12       1.00      1.00      1.00        35
          13       1.00      1.00      1.00  